In [1]:
import tensorflow as tf
print(tf.__version__)
import cv2
print(cv2.__version__)
import time
import os
import numpy as np
import posenet
from collections import deque

c:\users\micha\appdata\local\programs\python\python36\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


1.12.0
3.4.5


# Function for calculating angle between two vectors

In [2]:

def unit_vector(vector):
    """ Returns the unit vector of the vector.  """
    return vector / np.linalg.norm(vector)

def angle(v1, v2):
    v1_u = unit_vector(v1)
    v2_u = unit_vector(v2)
    return 180*(np.arccos(np.clip(np.dot(v1_u, v2_u), -1.0, 1.0)))/np.pi

In [3]:
def findMax(q): #Finds the maximum height
	if q[3] < q[0] and q[3] < q[1] and q[3] < q[2] and q[3] < q[4] and q[3] < q[5] and q[3] < q[6]:
		return q[3]
	else:
		return 0

def findMin(q): #Finds the minimum height
	if q[3] > q[0] and q[3] > q[1] and q[3] > q[2] and q[3] > q[4] and q[3] > q[5] and q[3] > q[6]:
		return q[3]
	else:
		return 0


In [4]:
def compareTolerance(list_current, average_prev, tolerance):
    average_current = np.mean(np.asarray(list_current))
    difference = np.abs(average_current - average_prev)
    if difference < tolerance*average_current:
        return False
    else:
        return True
    

In [5]:
def compareTolerance2(average_current, average_prev, tolerance):
    difference = np.abs(average_current - average_prev)
    if difference < tolerance:
        return True
    else:
        return False
    

# Parameters

In [6]:
path_input = './exercises'
path_output = './output'
print_min = True
model_number = 101
scale_factor = 0.2
delay = 3
exercise = "pullup2"
video_format = ".avi"
font_face = cv2.FONT_HERSHEY_SIMPLEX
font_scale = 0.5
font_color = (255,255,255)
body_points = {'rightShoulder':None,'rightWrist':None,'rightElbow':None,'leftShoulder':None,'leftWrist':None,'leftElbow':None,'rightHip':None,'leftHip':None,'rightKnee':None,'leftKnee':None}
body_points_score = {'rightShoulder':None,'rightWrist':None,'rightElbow':None,'leftShoulder':None,'leftWrist':None,'leftElbow':None,'rightHip':None,'leftHip':None,'rightKnee':None,'leftKnee':None}
body_angles = {'rightAngle':None,'leftAngle':None,'avgAngle':None}

# Image Analysis

In [46]:
filenames = [f.path for f in os.scandir(path_input) if f.is_file() and f.path.endswith(('.png', '.jpg'))]

start = time.time()
with tf.Session() as sess:
    model_cfg, model_outputs = posenet.load_model(model_number, sess)
    output_stride = model_cfg['output_stride']
    for f in filenames:
        print(f)
        input_image, draw_image, output_scale = posenet.read_imgfile(
            f, scale_factor=scale_factor, output_stride=output_stride)
        heatmaps_result, offsets_result, displacement_fwd_result, displacement_bwd_result = sess.run(
            model_outputs,
            feed_dict={'image:0': input_image}
        )

        pose_scores, keypoint_scores, keypoint_coords = posenet.decode_multiple_poses(
            heatmaps_result.squeeze(axis=0),
            offsets_result.squeeze(axis=0),
            displacement_fwd_result.squeeze(axis=0),
            displacement_bwd_result.squeeze(axis=0),
            output_stride=output_stride,
            max_pose_detections=10,
            min_pose_score=0.25)

        keypoint_coords *= output_scale


        draw_image = posenet.draw_skel_and_kp(
            draw_image, pose_scores, keypoint_scores, keypoint_coords,
            min_pose_score=0.25, min_part_score=0.25)

        cv2.imwrite(os.path.join(path_output, os.path.relpath(f, path_input)), draw_image)

        
        print("Results for image: %s" % f)
        for pi in range(len(pose_scores)):
            if pose_scores[pi] == 0.:
                break
            print('Pose #%d, score = %f' % (pi, pose_scores[pi]))
            for ki, (s, c) in enumerate(zip(keypoint_scores[pi, :], keypoint_coords[pi, :, :])):
                #print('Keypoint %s, score = %f, coord = %s' % (posenet.PART_NAMES[ki], s, c))
                if posenet.PART_NAMES[ki] in body_points:
                    body_points[posenet.PART_NAMES[ki]] = c
                    body_points_score[posenet.PART_NAMES[ki]] = s
        #Calculate angle between shoulder and wrist using the elbow as the origin
        angle_right = angle((body_points['rightShoulder']-body_points['rightElbow']),(body_points['rightWrist']-body_points['rightElbow']))
        angle_left = angle((body_points['leftShoulder']-body_points['leftElbow']),(body_points['leftWrist']-body_points['leftElbow']))
        
        #Calculate the average confidence score of the shoulder, elbow, and wrist
        confidence_right = body_points_score['rightShoulder'] + body_points_score['rightElbow'] + body_points_score['rightWrist']
        confidence_left = body_points_score['leftShoulder'] + body_points_score['leftElbow'] + body_points_score['leftWrist']
        confidence_right /= 3
        confidence_left /= 3
        
        print('Angle Right: %f degrees with confidence %f' %(angle_right,confidence_right))
        print('Angle Left: %f degrees with confidence %f' %(angle_left,confidence_left))
        print()
    print('Average FPS:', len(filenames) / (time.time() - start))


./exercises\proper_squat.png
Results for image: ./exercises\proper_squat.png


TypeError: unsupported operand type(s) for -: 'NoneType' and 'NoneType'

In [7]:
from tkinter import filedialog
from tkinter import *
from tkinter import ttk

root = Tk()
returnOpenFile = ""

def openFile():
  root.filename =  filedialog.askopenfilename(initialdir = "C:/Users/micha/Desktop/113dbspring19-master",title = "Select file",filetypes = (("avi files","*.avi"),("all files","*.*")))
  returnOpenFile = ""+root.filename
  return returnOpenFile

# Video Analysis for Pullups

In [10]:
exercise = "pullup2"
video_format = ".avi"
with tf.Session() as sess:
        model_cfg, model_outputs = posenet.load_model(model_number, sess)
        output_stride = model_cfg['output_stride']
        fileUploaded = openFile()
        print(fileUploaded)
        index = 0
        for char in reversed(fileUploaded):
            if(char == '/'):
                break
            index += 1
        fileUploaded = fileUploaded[(-1)*index:]
        print(fileUploaded)
        #cap = cv2.VideoCapture(exercise+video_format)
        cap = cv2.VideoCapture(fileUploaded)
        fourcc = cv2.VideoWriter_fourcc('X','V','I','D')
        vid_out = cv2.VideoWriter('output '+exercise+video_format,fourcc, 30, (600,900), True)
        cap.set(3, 1080)
        cap.set(4, 1920)
        
        #create other queues to get more info
        moreInfoQ = deque()
        imageQ = deque()
        infoQ = deque()
        start = time.time()
        frame_count = 0
        picture_count = 1
        delay_count = 0
        avg_count = 10
        avg_counter = 0
        threshold = 0
        maxima_count = 0 
        reps = 0
        next_maxima = 'N/A'
        rightWrist = 0
        rightShoulder = 0
        min_difference = 0
        max_difference = 0
        
        
        
        
        # Variables for tracking maxima
        m_var = {'Previous Average': 0, 'Initial Position': 0, 'Queue': deque(), 'First': True, 'Max Found': False,'Maxima Text':''}
        body = {'Left Eye':m_var}
        tolerance = 0.2
        tolerance2 = 50
        
        c_r = deque()
        c_l = deque()
        # maximum angles
        st = []
        # minimum angles
        sb = []
        while True:
            res, img = cap.read()
            if not res:
                break
            else:
                input_image, display_image, output_scale = posenet.process_input(img, scale_factor, output_stride)
            info_dict = {"Body Points": None, "Body Points Score": None, "Body Angles":None, "Scores Right": None, "Scores Left": None, "Image": None}
        
           
            #input_image, display_image, output_scale = posenet.read_cap(
              #  cap, scale_factor=0.2, output_stride=output_stride)
            
            heatmaps_result, offsets_result, displacement_fwd_result, displacement_bwd_result = sess.run(
                model_outputs,
                feed_dict={'image:0': input_image}
            )

            pose_scores, keypoint_scores, keypoint_coords = posenet.decode_multi.decode_multiple_poses(
                heatmaps_result.squeeze(axis=0),
                offsets_result.squeeze(axis=0),
                displacement_fwd_result.squeeze(axis=0),
                displacement_bwd_result.squeeze(axis=0),
                output_stride=output_stride,
                max_pose_detections=10,
                min_pose_score=0.15)

            keypoint_coords *= output_scale

            # TODO this isn't particularly fast, use GL for drawing and display someday...
            overlay_image = posenet.draw_skel_and_kp(
                display_image, pose_scores, keypoint_scores, keypoint_coords,
                min_pose_score=0, min_part_score=0)
            
            # CHANGE THE RESOLUTION OF THE OUTPUT VIDEO
            #overlay_image = cv2.resize(overlay_image, (1920, 1080))
            overlay_image = cv2.resize(overlay_image, (600, 900))
            
            # RESET VARIABLES FOR FOUND MAXIMA FOR EACH PART
            for body_part in body:
                body[body_part]['Max Found'] = False
            maxima_found = False
            
            # APPEND THE IMAGE TO THE IMAGEQ
            imageQ.append(overlay_image)
            for pi in range(len(pose_scores)):
                if pose_scores[pi] == 0.:
                    break
                #logging.warning('Pose #%d, score = %f' % (pi, pose_scores[pi]))
                for ki, (s, c) in enumerate(zip(keypoint_scores[pi, :], keypoint_coords[pi, :, :])):
                    if posenet.PART_NAMES[ki] == "rightWrist":
                        rightWrist = c[0]
                    if posenet.PART_NAMES[ki] == "rightShoulder":
                        rightShoulder = c[0]
                    if posenet.PART_NAMES[ki] == "leftEye":
                        #print('Keypoint %s, score = %f, coord = %s' % (posenet.PART_NAMES[ki], s, c))
                        #adding coordinate to running queue
                        body['Left Eye']['Queue'].append(c[0])
                        #append more infomation to other queues here
                        
                        #if length equals 7, then will see if mid point is the min
                        if body['Left Eye']['Initial Position'] == 0:
                            body['Left Eye']['Initial Position'] = c[0]
                        if len(body['Left Eye']['Queue']) == 7:
                            if findMin(body['Left Eye']['Queue']) != 0 and compareTolerance(body['Left Eye']['Queue'],body['Left Eye']['Previous Average'], tolerance):
                                if body['Left Eye']['First'] == False or (body['Left Eye']['First'] == True and body['Left Eye']['Initial Position'] < c[0]):
                                    body['Left Eye']['Maxima Text'] = 'Minimum'
                                    body['Left Eye']['Previous Average'] = np.mean(np.asarray(body['Left Eye']['Queue']))
                                    body['Left Eye']['First'] = False
                                    body['Left Eye']['Max Found'] = True
                                    print('eye min')
                            if findMax(body['Left Eye']['Queue']) != 0 and compareTolerance(body['Left Eye']['Queue'],body['Left Eye']['Previous Average'], tolerance):
                                if body['Left Eye']['First'] == False or (body['Left Eye']['First'] == True and body['Left Eye']['Initial Position'] > c[0]):
                                    body['Left Eye']['Maxima Text'] = 'Maximum'
                                    body['Left Eye']['Previous Average'] = np.mean(np.asarray(body['Left Eye']['Queue']))
                                    body['Left Eye']['First'] = False
                                    body['Left Eye']['Max Found'] = True
                                    print('eye max')
                            body['Left Eye']['Queue'].popleft()                    
                    body_points[posenet.PART_NAMES[ki]] = c
                    body_points_score[posenet.PART_NAMES[ki]] = s
            
            # Check if there is a maxima in any of the points found
    
            
            #Calculate the average confidence score of the shoulder, elbow, and wrist
            confidence_right = body_points_score['rightShoulder'] + body_points_score['rightElbow'] + body_points_score['rightWrist']
            confidence_left = body_points_score['leftShoulder'] + body_points_score['leftElbow'] + body_points_score['leftWrist']
            confidence_right /= 3
            confidence_left /= 3
            angle_right = angle((body_points['rightShoulder']-body_points['rightElbow']),(body_points['rightWrist']-body_points['rightElbow']))
            angle_left = angle((body_points['leftShoulder']-body_points['leftElbow']),(body_points['leftWrist']-body_points['leftElbow']))
            weight_r = confidence_right / (confidence_right + confidence_left)
            weight_l = confidence_left / (confidence_right + confidence_left)
            weighted_average = (weight_r*angle_right + weight_l*angle_left)       
            body_angles['rightAngle'] = angle_right
            body_angles['leftAngle'] = angle_left
            body_angles['avgAngle'] = weighted_average
            difference = np.abs(rightShoulder - rightWrist)
            
            
            if delay_count < delay:
                infoQ.append(info_dict)
                delay_count+=1
            else:
                oldQ = infoQ.popleft()
                infoQ.append(info_dict)
                
            if avg_counter < avg_count:    
                c_r.append(confidence_right)
                c_l.append(confidence_left)
                avg_counter += 1
            else:
                c_r.popleft()
                c_l.popleft()
                c_r.append(confidence_right)
                c_l.append(confidence_left)
            
            # MAKE IT SO WE CAN ONLY HAVE ALTERNATING MAXIMUM AND MINIMUM
            for body_part in body:
                if body[body_part]['Max Found'] == True and maxima_found == False:
                    print(difference)
                    if max_difference != 0 and compareTolerance2(max_difference, difference, tolerance2):
                        body[body_part]['Maxima Text'] = 'Maximum'
                    if min_difference != 0 and compareTolerance2(min_difference, difference, tolerance2):
                        body[body_part]['Maxima Text'] = 'Minimum'
                    if next_maxima == 'N/A' or body[body_part]['Maxima Text'] != next_maxima:
                        next_maxima = body[body_part]['Maxima Text']
                        maxima_found = True
            
            # Create video
            data_text_r = str('Angle Right: %f degrees with confidence %f' %(angle_right,confidence_right))
            data_text_l = str('Angle Left: %f degrees with confidence %f' %(angle_left,confidence_left))
            data_text_avg = str('Average Angle: %f' %(weighted_average))
            m_text = 'Number of maxima = %i ' %(picture_count-1)
            # Creating text to overlay data over the video
            #overlay_image = cv2.rectangle(overlay_image, (0, 700), (600, 500),(0, 0, 0), -1)
            overlay_image = cv2.putText(overlay_image, data_text_r, (50,650), font_face, font_scale, font_color)
            overlay_image = cv2.putText(overlay_image, data_text_l, (50,700), font_face, font_scale, font_color)
            overlay_image = cv2.putText(overlay_image, data_text_avg, (50,750), font_face, font_scale, font_color)
            overlay_image = cv2.putText(overlay_image, 'Pullup Count: ' + str(maxima_count), (50,550), font_face, font_scale, font_color)
            vid_out.write(overlay_image)
            
            # STORE INFORMATION INTO DICTIONARY
            info_dict['Body Points'] = body_points
            info_dict['Body Points Score'] = body_points_score
            info_dict['Body Angles'] = body_angles
            info_dict['Scores Right'] = confidence_right
            info_dict['Scores Left'] = confidence_left
            info_dict['Image'] = overlay_image
            
            if print_min and maxima_found: 
                avg_r = 0
                avg_l = 0
                for i in np.arange(len(c_r)):
                    avg_r += c_r[i]
                    avg_l += c_l[i]
                avg_r /= avg_count
                avg_l /= avg_count
                if (avg_r > threshold and avg_l > threshold) or (oldQ['Scores Right'] > threshold and oldQ['Scores Left'] > threshold):
                    #Calculate angle between shoulder and wrist using the elbow as the origin
                    #angle_right = angle((oldQ['Body Points']['rightShoulder']-oldQ['Body Points']['rightElbow']),(oldQ['Body Points']['rightWrist']-oldQ['Body Points']['rightElbow']))
                    #angle_left = angle((oldQ['Body Points']['leftShoulder']-oldQ['Body Points']['leftElbow']),(oldQ['Body Points']['leftWrist']-oldQ['Body Points']['leftElbow']))
                    if next_maxima == 'Maximum':
                        st.append(angle_right)
                        st.append(angle_left)
                        maxima_count += 1
                        if max_difference == 0:
                            max_difference = difference
                    else:
                        sb.append(angle_right)
                        sb.append(angle_left)
                        if min_difference == 0:
                            min_difference = difference
                    print('For image number: ', picture_count)
                    print(next_maxima, ' Height Found at %f'%difference)
                    print('Angle Right: %f degrees with confidence %f and average confidence %f' %(oldQ['Body Angles']['rightAngle'],oldQ['Scores Right'],avg_r))
                    print('Angle Left: %f degrees with confidence %f and average confidence %f' %(oldQ['Body Angles']['leftAngle'],oldQ['Scores Left'],avg_l))
                    print()
                    oldQ['Image'] = cv2.putText(oldQ['Image'], next_maxima, (50,600), font_face, font_scale, font_color)
                    cv2.imwrite(r'C:\Users\micha\Desktop\ML\113dbspring19\posenet-python-master\posenet-python-master\output\ ' + next_maxima + ' ' + exercise + ' ' +  str(picture_count) + '.jpg',oldQ['Image'])
                    picture_count += 1
            
            cv2.imshow('posenet', overlay_image)
            frame_count += 1
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
                
        st = np.asarray(st)
        sb = np.asarray(sb)
        print('The maximum angle average is %f and the standard deviation is %f'% (np.mean(st),np.std(st)))
        print('The minimum angle average is %f and the standard deviation is %f'% (np.mean(sb),np.std(sb)))
        print(max_difference)
        print(min_difference)
        vid_out.release()
        cap.release()
        cv2.destroyAllWindows();
        print('Average FPS: ', frame_count / (time.time() - start))

C:\Users\micha\Desktop\ML\113dbspring19\posenet-python-master\posenet-python-master\posenet\converter\config.py:9: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  cfg = yaml.load(cfg_f)


C:/Users/micha/Desktop/ML/113dbspring19/posenet-python-master/posenet-python-master/jordanpullup.avi
jordanpullup.avi
eye min
272.64394500376187
For image number:  1
Minimum  Height Found at 272.643945
Angle Right: 163.980786 degrees with confidence 0.570765 and average confidence 0.624438
Angle Left: 158.080200 degrees with confidence 0.499599 and average confidence 0.509491

eye min
21.653570627887916
eye min
264.3267656022009
eye max
88.91253163378519
For image number:  2
Maximum  Height Found at 88.912532
Angle Right: 138.033826 degrees with confidence 0.310059 and average confidence 0.380175
Angle Left: 162.522624 degrees with confidence 0.246097 and average confidence 0.376605

eye min
202.91622651690182
For image number:  3
Minimum  Height Found at 202.916227
Angle Right: 115.865086 degrees with confidence 0.373304 and average confidence 0.494074
Angle Left: 143.838005 degrees with confidence 0.557013 and average confidence 0.532719

eye max
13.71195396096789
For image number:  

# Video Analysis for Pushups

In [54]:
exercise = "pushup2"
video_format = ".avi"
with tf.Session() as sess:
        model_cfg, model_outputs = posenet.load_model(model_number, sess)
        output_stride = model_cfg['output_stride']
        cap = cv2.VideoCapture(exercise+video_format)
        fourcc = cv2.VideoWriter_fourcc('X','V','I','D')
        vid_out = cv2.VideoWriter('output '+exercise+video_format,fourcc, 30, (600,900), True)
        cap.set(3, 1080)
        cap.set(4, 1920)
        
        #create other queues to get more info
        moreInfoQ = deque()
        imageQ = deque()
        infoQ = deque()
        start = time.time()
        frame_count = 0
        picture_count = 1
        delay_count = 0
        avg_count = 10
        avg_counter = 0
        threshold = 0
        maxima_count = 0 
        reps = 0
        next_maxima = 'N/A'
        rightElbow = 0
        rightShoulder = 0
        min_difference = 0
        max_difference = 0
        reclassify = False
        
        
        
        # Variables for tracking maxima
        m_var = {'Previous Average': 0, 'Initial Position': 0, 'Queue': deque(), 'First': True, 'Max Found': False,'Maxima Text':''}
        body = {'Left Eye':m_var}
        tolerance = 0.2
        tolerance2 = 20
        
        c_r = deque()
        c_l = deque()
        # maximum angles
        st = []
        # minimum angles
        sb = []
        while True:
            res, img = cap.read()
            if not res:
                break
            else:
                input_image, display_image, output_scale = posenet.process_input(img, scale_factor, output_stride)
            info_dict = {"Body Points": None, "Body Points Score": None, "Body Angles":None, "Scores Right": None, "Scores Left": None, "Image": None}
        
           
            #input_image, display_image, output_scale = posenet.read_cap(
              #  cap, scale_factor=0.2, output_stride=output_stride)
            
            heatmaps_result, offsets_result, displacement_fwd_result, displacement_bwd_result = sess.run(
                model_outputs,
                feed_dict={'image:0': input_image}
            )

            pose_scores, keypoint_scores, keypoint_coords = posenet.decode_multi.decode_multiple_poses(
                heatmaps_result.squeeze(axis=0),
                offsets_result.squeeze(axis=0),
                displacement_fwd_result.squeeze(axis=0),
                displacement_bwd_result.squeeze(axis=0),
                output_stride=output_stride,
                max_pose_detections=10,
                min_pose_score=0.15)

            keypoint_coords *= output_scale

            # TODO this isn't particularly fast, use GL for drawing and display someday...
            overlay_image = posenet.draw_skel_and_kp(
                display_image, pose_scores, keypoint_scores, keypoint_coords,
                min_pose_score=0, min_part_score=0)
            
            # CHANGE THE RESOLUTION OF THE OUTPUT VIDEO
            #overlay_image = cv2.resize(overlay_image, (1920, 1080))
            overlay_image = cv2.resize(overlay_image, (600, 900))
            
            # RESET VARIABLES FOR FOUND MAXIMA FOR EACH PART
            for body_part in body:
                body[body_part]['Max Found'] = False
            maxima_found = False
            
            # APPEND THE IMAGE TO THE IMAGEQ
            imageQ.append(overlay_image)
            for pi in range(len(pose_scores)):
                if pose_scores[pi] == 0.:
                    break
                #logging.warning('Pose #%d, score = %f' % (pi, pose_scores[pi]))
                for ki, (s, c) in enumerate(zip(keypoint_scores[pi, :], keypoint_coords[pi, :, :])):
                    if posenet.PART_NAMES[ki] == "rightElbow":
                        rightElbow = c[0]
                    if posenet.PART_NAMES[ki] == "rightShoulder":
                        rightShoulder = c[0]
                    if posenet.PART_NAMES[ki] == "leftShoulder":
                        #print('Keypoint %s, score = %f, coord = %s' % (posenet.PART_NAMES[ki], s, c))
                        #adding coordinate to running queue
                        body['Left Eye']['Queue'].append(c[0])
                        #append more infomation to other queues here
                        
                        #if length equals 7, then will see if mid point is the min
                        if body['Left Eye']['Initial Position'] == 0:
                            body['Left Eye']['Initial Position'] = c[0]
                        if len(body['Left Eye']['Queue']) == 7:
                            if findMin(body['Left Eye']['Queue']) != 0 and compareTolerance(body['Left Eye']['Queue'],body['Left Eye']['Previous Average'], tolerance):
                                if body['Left Eye']['First'] == False or (body['Left Eye']['First'] == True and body['Left Eye']['Initial Position'] < c[0]):
                                    body['Left Eye']['Maxima Text'] = 'Minimum'
                                    body['Left Eye']['Previous Average'] = np.mean(np.asarray(body['Left Eye']['Queue']))
                                    body['Left Eye']['First'] = False
                                    body['Left Eye']['Max Found'] = True
                                    print('eye min')
                            if findMax(body['Left Eye']['Queue']) != 0 and compareTolerance(body['Left Eye']['Queue'],body['Left Eye']['Previous Average'], tolerance):
                                if body['Left Eye']['First'] == False or (body['Left Eye']['First'] == True and body['Left Eye']['Initial Position'] > c[0]):
                                    body['Left Eye']['Maxima Text'] = 'Maximum'
                                    body['Left Eye']['Previous Average'] = np.mean(np.asarray(body['Left Eye']['Queue']))
                                    body['Left Eye']['First'] = False
                                    body['Left Eye']['Max Found'] = True
                                    print('eye max')
                            body['Left Eye']['Queue'].popleft()                    
                    body_points[posenet.PART_NAMES[ki]] = c
                    body_points_score[posenet.PART_NAMES[ki]] = s
            
            # Check if there is a maxima in any of the points found
    
            
            #Calculate the average confidence score of the shoulder, elbow, and wrist
            confidence_right = body_points_score['rightShoulder'] + body_points_score['rightElbow'] + body_points_score['rightWrist']
            confidence_left = body_points_score['leftShoulder'] + body_points_score['leftElbow'] + body_points_score['leftWrist']
            confidence_right /= 3
            confidence_left /= 3
            angle_right = angle((body_points['rightShoulder']-body_points['rightElbow']),(body_points['rightWrist']-body_points['rightElbow']))
            angle_left = angle((body_points['leftShoulder']-body_points['leftElbow']),(body_points['leftWrist']-body_points['leftElbow']))
            weight_r = confidence_right / (confidence_right + confidence_left)
            weight_l = confidence_left / (confidence_right + confidence_left)
            weighted_average = (weight_r*angle_right + weight_l*angle_left)       
            body_angles['rightAngle'] = angle_right
            body_angles['leftAngle'] = angle_left
            body_angles['avgAngle'] = weighted_average
            difference = np.abs(rightShoulder - rightElbow)
            
            
            if delay_count < delay:
                infoQ.append(info_dict)
                delay_count+=1
            else:
                oldQ = infoQ.popleft()
                infoQ.append(info_dict)
                
            if avg_counter < avg_count:    
                c_r.append(confidence_right)
                c_l.append(confidence_left)
                avg_counter += 1
            else:
                c_r.popleft()
                c_l.popleft()
                c_r.append(confidence_right)
                c_l.append(confidence_left)
            
            # MAKE IT SO WE CAN ONLY HAVE ALTERNATING MAXIMUM AND MINIMUM
            for body_part in body:
                if body[body_part]['Max Found'] == True and maxima_found == False:
                    print(difference)
                    if max_difference != 0 and compareTolerance2(max_difference, difference, tolerance2):
                        body[body_part]['Maxima Text'] = 'Maximum'
                    if min_difference != 0 and compareTolerance2(min_difference, difference, tolerance2):
                        body[body_part]['Maxima Text'] = 'Minimum'
                    if next_maxima == 'N/A' or body[body_part]['Maxima Text'] != next_maxima:
                        next_maxima = body[body_part]['Maxima Text']
                        maxima_found = True
            
            # Create video
            data_text_r = str('Angle Right: %f degrees with confidence %f' %(angle_right,confidence_right))
            data_text_l = str('Angle Left: %f degrees with confidence %f' %(angle_left,confidence_left))
            data_text_avg = str('Average Angle: %f' %(weighted_average))
            m_text = 'Number of maxima = %i ' %(picture_count-1)
            overlay_image = cv2.putText(overlay_image, data_text_r, (50,650), font_face, font_scale, font_color)
            overlay_image = cv2.putText(overlay_image, data_text_l, (50,700), font_face, font_scale, font_color)
            overlay_image = cv2.putText(overlay_image, data_text_avg, (50,750), font_face, font_scale, font_color)
            overlay_image = cv2.putText(overlay_image, 'Pushup Count: ' + str(maxima_count), (50,550), font_face, font_scale, font_color)
            vid_out.write(overlay_image)
            
            # STORE INFORMATION INTO DICTIONARY
            info_dict['Body Points'] = body_points
            info_dict['Body Points Score'] = body_points_score
            info_dict['Body Angles'] = body_angles
            info_dict['Scores Right'] = confidence_right
            info_dict['Scores Left'] = confidence_left
            info_dict['Image'] = overlay_image
            
            if print_min and maxima_found: 
                avg_r = 0
                avg_l = 0
                for i in np.arange(len(c_r)):
                    avg_r += c_r[i]
                    avg_l += c_l[i]
                avg_r /= avg_count
                avg_l /= avg_count
                if (avg_r > threshold and avg_l > threshold) or (oldQ['Scores Right'] > threshold and oldQ['Scores Left'] > threshold):
                    #Calculate angle between shoulder and wrist using the elbow as the origin
                    #angle_right = angle((oldQ['Body Points']['rightShoulder']-oldQ['Body Points']['rightElbow']),(oldQ['Body Points']['rightWrist']-oldQ['Body Points']['rightElbow']))
                    #angle_left = angle((oldQ['Body Points']['leftShoulder']-oldQ['Body Points']['leftElbow']),(oldQ['Body Points']['leftWrist']-oldQ['Body Points']['leftElbow']))
                    if next_maxima == 'Maximum':
                        st.append(angle_right)
                        st.append(angle_left)
                        if max_difference == 0:
                            max_difference = difference
                    else:
                        maxima_count += 1
                        sb.append(angle_right)
                        sb.append(angle_left)
                        if min_difference == 0:
                            min_difference = difference
                    print('For image number: ', picture_count)
                    print(next_maxima, ' Height Found at %f'%difference)
                    print('Angle Right: %f degrees with confidence %f and average confidence %f' %(oldQ['Body Angles']['rightAngle'],oldQ['Scores Right'],avg_r))
                    print('Angle Left: %f degrees with confidence %f and average confidence %f' %(oldQ['Body Angles']['leftAngle'],oldQ['Scores Left'],avg_l))
                    print()
                    oldQ['Image'] = cv2.putText(oldQ['Image'], next_maxima, (50,600), font_face, font_scale, font_color)
                    cv2.imwrite(r'C:\Users\micha\Desktop\ML\113dbspring19\posenet-python-master\posenet-python-master\output\ ' + next_maxima + ' ' + exercise + ' ' +  str(picture_count) + '.jpg',oldQ['Image'])
                    picture_count += 1
            
            cv2.imshow('posenet', overlay_image)
            frame_count += 1
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
                
        st = np.asarray(st)
        sb = np.asarray(sb)
        print('The maximum angle average is %f and the standard deviation is %f'% (np.mean(st),np.std(st)))
        print('The minimum angle average is %f and the standard deviation is %f'% (np.mean(sb),np.std(sb)))
        print(max_difference)
        print(min_difference)
        vid_out.release()
        cap.release()
        cv2.destroyAllWindows();
        print('Average FPS: ', frame_count / (time.time() - start))

eye min
264.0220713429636
For image number:  1
Minimum  Height Found at 264.022071
Angle Right: 175.901789 degrees with confidence 0.858676 and average confidence 0.864077
Angle Left: 172.990766 degrees with confidence 0.838634 and average confidence 0.763686

eye min
92.03050410283083
eye max
268.0807669503348
eye min
128.03401996563014
eye max
231.24309272270693
For image number:  2
Maximum  Height Found at 231.243093
Angle Right: 153.854077 degrees with confidence 0.789835 and average confidence 0.821183
Angle Left: 163.111816 degrees with confidence 0.737873 and average confidence 0.827445

eye min
97.23567625764122
For image number:  3
Minimum  Height Found at 97.235676
Angle Right: 139.739490 degrees with confidence 0.583192 and average confidence 0.611782
Angle Left: 117.705543 degrees with confidence 0.191835 and average confidence 0.492963

eye max
217.41909116274348
For image number:  4
Maximum  Height Found at 217.419091
Angle Right: 156.253545 degrees with confidence 0.8402

# Video Analysis for Squats

In [35]:
exercise = "squat_side"
video_format = ".avi"
with tf.Session() as sess:
        model_cfg, model_outputs = posenet.load_model(model_number, sess)
        output_stride = model_cfg['output_stride']
        cap = cv2.VideoCapture(exercise+video_format)
        fourcc = cv2.VideoWriter_fourcc('X','V','I','D')
        vid_out = cv2.VideoWriter('output '+exercise+video_format,fourcc, 30, (600,900), True)
        cap.set(3, 1080)
        cap.set(4, 1920)
        
        #create other queues to get more info
        moreInfoQ = deque()
        imageQ = deque()
        infoQ = deque()
        start = time.time()
        frame_count = 0
        picture_count = 1
        delay_count = 0
        avg_count = 10
        avg_counter = 0
        threshold = 0
        maxima_count = 0 
        reps = 0
        next_maxima = 'N/A'
        rightHip = 0
        rightKnee = 0
        min_difference = 0
        max_difference = 0
        correct = 0
        incorrect = 0
        
        
        
        
        # Variables for tracking maxima
        m_var = {'Previous Average': 0, 'Initial Position': 0, 'Queue': deque(), 'First': True, 'Max Found': False,'Maxima Text':''}
        body = {'Left Eye':m_var}
        tolerance = 0.2
        tolerance2 = 50
        
        c_r = deque()
        c_l = deque()
        # hips past knees
        hips = []
        # maximum angles
        st = []
        # minimum angles
        sb = []
        while True:
            res, img = cap.read()
            if not res:
                break
            else:
                input_image, display_image, output_scale = posenet.process_input(img, scale_factor, output_stride)
            info_dict = {"Body Points": None, "Body Points Score": None, "Body Angles":None, "Scores Right": None, "Scores Left": None, "Image": None}
        
           
            #input_image, display_image, output_scale = posenet.read_cap(
              #  cap, scale_factor=0.2, output_stride=output_stride)
            
            heatmaps_result, offsets_result, displacement_fwd_result, displacement_bwd_result = sess.run(
                model_outputs,
                feed_dict={'image:0': input_image}
            )

            pose_scores, keypoint_scores, keypoint_coords = posenet.decode_multi.decode_multiple_poses(
                heatmaps_result.squeeze(axis=0),
                offsets_result.squeeze(axis=0),
                displacement_fwd_result.squeeze(axis=0),
                displacement_bwd_result.squeeze(axis=0),
                output_stride=output_stride,
                max_pose_detections=10,
                min_pose_score=0.15)

            keypoint_coords *= output_scale

            # TODO this isn't particularly fast, use GL for drawing and display someday...
            overlay_image = posenet.draw_skel_and_kp(
                display_image, pose_scores, keypoint_scores, keypoint_coords,
                min_pose_score=0, min_part_score=0)
            
            # CHANGE THE RESOLUTION OF THE OUTPUT VIDEO
            #overlay_image = cv2.resize(overlay_image, (1920, 1080))
            overlay_image = cv2.resize(overlay_image, (600, 900))
            
            # RESET VARIABLES FOR FOUND MAXIMA FOR EACH PART
            for body_part in body:
                body[body_part]['Max Found'] = False
            maxima_found = False
            
            # APPEND THE IMAGE TO THE IMAGEQ
            imageQ.append(overlay_image)
            for pi in range(len(pose_scores)):
                if pose_scores[pi] == 0.:
                    break
                #logging.warning('Pose #%d, score = %f' % (pi, pose_scores[pi]))
                for ki, (s, c) in enumerate(zip(keypoint_scores[pi, :], keypoint_coords[pi, :, :])):
                    if posenet.PART_NAMES[ki] == "rightHip":
                        rightHip = c[0]
                    if posenet.PART_NAMES[ki] == "rightKnee":
                        rightKnee = c[0]
                    if posenet.PART_NAMES[ki] == "rightShoulder":
                        #print('Keypoint %s, score = %f, coord = %s' % (posenet.PART_NAMES[ki], s, c))
                        #adding coordinate to running queue
                        body['Left Eye']['Queue'].append(c[0])
                        #append more infomation to other queues here
                        
                        #if length equals 7, then will see if mid point is the min
                        if body['Left Eye']['Initial Position'] == 0:
                            body['Left Eye']['Initial Position'] = c[0]
                        if len(body['Left Eye']['Queue']) == 7:
                            if findMin(body['Left Eye']['Queue']) != 0 and compareTolerance(body['Left Eye']['Queue'],body['Left Eye']['Previous Average'], tolerance):
                                if body['Left Eye']['First'] == False or (body['Left Eye']['First'] == True and body['Left Eye']['Initial Position'] < c[0]):
                                    body['Left Eye']['Maxima Text'] = 'Minimum'
                                    body['Left Eye']['Previous Average'] = np.mean(np.asarray(body['Left Eye']['Queue']))
                                    body['Left Eye']['First'] = False
                                    body['Left Eye']['Max Found'] = True
                                    print('eye min')
                            if findMax(body['Left Eye']['Queue']) != 0 and compareTolerance(body['Left Eye']['Queue'],body['Left Eye']['Previous Average'], tolerance):
                                if body['Left Eye']['First'] == False or (body['Left Eye']['First'] == True and body['Left Eye']['Initial Position'] > c[0]):
                                    body['Left Eye']['Maxima Text'] = 'Maximum'
                                    body['Left Eye']['Previous Average'] = np.mean(np.asarray(body['Left Eye']['Queue']))
                                    body['Left Eye']['First'] = False
                                    body['Left Eye']['Max Found'] = True
                                    print('eye max')
                            body['Left Eye']['Queue'].popleft()                    
                    body_points[posenet.PART_NAMES[ki]] = c
                    body_points_score[posenet.PART_NAMES[ki]] = s
            
            # Check if there is a maxima in any of the points found
    
            
            #Calculate the average confidence score of the shoulder, elbow, and wrist
            difference = np.abs(rightKnee - rightHip)
            
            
            if delay_count < delay:
                infoQ.append(info_dict)
                delay_count+=1
            else:
                oldQ = infoQ.popleft()
                infoQ.append(info_dict)
                
            if avg_counter < avg_count:    
                c_r.append(confidence_right)
                c_l.append(confidence_left)
                avg_counter += 1
            else:
                c_r.popleft()
                c_l.popleft()
                c_r.append(confidence_right)
                c_l.append(confidence_left)
            
            # MAKE IT SO WE CAN ONLY HAVE ALTERNATING MAXIMUM AND MINIMUM
            for body_part in body:
                if body[body_part]['Max Found'] == True and maxima_found == False:
                    print(difference)
                    if max_difference != 0 and compareTolerance2(max_difference, difference, tolerance2):
                        body[body_part]['Maxima Text'] = 'Maximum'
                    if min_difference != 0 and compareTolerance2(min_difference, difference, tolerance2):
                        body[body_part]['Maxima Text'] = 'Minimum'
                    if next_maxima == 'N/A' or body[body_part]['Maxima Text'] != next_maxima:
                        next_maxima = body[body_part]['Maxima Text']
                        maxima_found = True
            
            # Create video

            mc_text = 'Number of correct squats = %i ' %(correct)
            mw_text = 'Number of incorrect squats = %i ' %(incorrect)
            overlay_image = cv2.putText(overlay_image, mc_text, (50,650), font_face, font_scale, font_color)
            overlay_image = cv2.putText(overlay_image, mw_text, (50,700), font_face, font_scale, font_color)
            #overlay_image = cv2.putText(overlay_image, str(maxima_count), (50,500), font_face, font_scale, font_color)
            vid_out.write(overlay_image)
            
            # STORE INFORMATION INTO DICTIONARY
            info_dict['Body Points'] = body_points
            info_dict['Image'] = overlay_image
            
            if print_min and maxima_found: 
                maxima_count += 1
                if next_maxima == 'Maximum':
                    if max_difference == 0:
                        max_difference = difference
                else:
                    if(oldQ['Body Points']['leftHip'][0] < oldQ['Body Points']['leftKnee'][0]*1.3 and oldQ['Body Points']['leftHip'][0] > oldQ['Body Points']['leftKnee'][0]*0.7):
                        hips.append(1)
                        correct += 1
                    else:
                        hips.append(0)
                        incorrect += 1
                    if min_difference == 0:
                        min_difference = difference
                print('For image number: ', picture_count)
                print(next_maxima, ' Height Found at %f'%difference)
                print()
                oldQ['Image'] = cv2.putText(oldQ['Image'], next_maxima, (50,600), font_face, font_scale, font_color)
                cv2.imwrite(r'C:\Users\micha\Desktop\ML\113dbspring19\posenet-python-master\posenet-python-master\output\ ' + next_maxima + ' ' + exercise + ' ' +  str(picture_count) + '.jpg',oldQ['Image'])
                picture_count += 1
            
            cv2.imshow('posenet', overlay_image)
            frame_count += 1
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break


        print(max_difference)
        print(min_difference)
        vid_out.release()
        cap.release()
        cv2.destroyAllWindows();
        print('Average FPS: ', frame_count / (time.time() - start))

eye max
290.7743260767553
For image number:  1
Maximum  Height Found at 290.774326

eye max
58.68822023466032
eye min
62.51280747450778
For image number:  2
Minimum  Height Found at 62.512807

eye max
309.85550441989653
For image number:  3
Maximum  Height Found at 309.855504

eye min
2.5333854378045544
For image number:  4
Minimum  Height Found at 2.533385

eye max
310.1474958147321
For image number:  5
Maximum  Height Found at 310.147496

290.7743260767553
62.51280747450778
Average FPS:  35.75328181593244
